In [38]:
"""
TODO:
Some of the 3D points along the road, wont produce
a skyline, reason is unkown.

Remove the first column of ones on the exported
csv-file of observation points.

Check if this solution works for road branches, not
just a single continous line.

"""

import arcpy

"""
The is ment to be a proof of concept script for the
study 

This script is ment 




"""

# Inputs for the arcgis enviroment: 6801_50m_33.dem, 6802_50m_33.dem, 6900_50m_33.dem, 6901_50m_33.dem, 6902_50m_33.dem
# - Terrain map: 
# - Road curve
# Output:
# - Observation points (long, lat, alt)
# - angles_table
DATA_DUMP_LOCATION = r'C:/Users/sinhav/Documents/Kode/GDOPER_sindre/datadump_from_arcgis/'
PROJECT_LOCATION   = r'C:/Users/sinhav/Documents/ArcGIS/Projects/GDOP_preprocessing_v1/'
PROJECT_DATABASE_LOCATION = f'{PROJECT_LOCATION}GDOP_preprocessing_v1.gdb'
arcpy.env.workspace = PROJECT_DATABASE_LOCATION

road_shp = 'E136_S8_17'
road_dis = 'E136_S8_17_dis'

# Merge all roadobjects into one
arcpy.management.Dissolve(road_shp, road_dis)

DEM = ['6801_50m_33.dem', '6802_50m_33.dem',
       '6900_50m_33.dem', '6901_50m_33.dem',
       '6901_50m_33.dem']

input_rasters = ''.join([x+";" for x in DEM])[:-1]
output_raster = r'dem50.tif'
print(input_rasters)

#arcpy.management.MosaicToNewRaster
#arcpy.management.MosaicToNewRaster
PROJECT_DATABASE_LOCATION = r'C:\Users\sinhav\Documents\ArcGIS\Projects\GDOP_preprocessing_v1'
arcpy.MosaicToNewRaster_management(input_rasters=input_rasters,
                                   output_location=PROJECT_DATABASE_LOCATION,
                                   raster_dataset_name_with_extension=output_raster,
                                   pixel_type="16_BIT_SIGNED",
                                   number_of_bands="1")


road_2Dpoints = 'E136_S8_17_pts_2D'
SPACING = 100 # meters

# Create spaced points along roadcurve
arcpy.management.GeneratePointsAlongLines(Input_Features=road_dis,
                                          Output_Feature_Class=road_2Dpoints,
                                          Point_Placement='DISTANCE',
                                          Distance=f'{SPACING} meters')

road_3Dpoints = r'E136_S8_17_pts_3D'
arcpy.ddd.InterpolateShape(in_surface=output_raster,
                           in_feature_class=road_2Dpoints,
                           out_feature_class=road_3Dpoints)

observation_points = r'observation_points'
WKID_GPS_coordinate_standard = 4326 # https://developers.arcgis.com/documentation/spatial-references/
out_coordinate_system = arcpy.SpatialReference(WKID_GPS_coordinate_standard)#, WKID_Z_WGS_1984)
arcpy.Project_management(road_3Dpoints, observation_points, out_coordinate_system)
arcpy.AddXY_management(observation_points)
arcpy.management.AlterField(observation_points, 'POINT_X', new_field_name='LONG', new_field_alias='LONG')
arcpy.management.AlterField(observation_points, 'POINT_Y', new_field_name='LAT',  new_field_alias='LAT')
arcpy.management.AlterField(observation_points, 'POINT_Z', new_field_name='ALT', new_field_alias='ALT')
arcpy.conversion.ExportTable(in_table=observation_points,
                             out_table=DATA_DUMP_LOCATION+'/'+observation_points+'.csv'
                            )
                             


AZIMUTH_RESOLUTION_DEGREES = 1
MAX_RADIUS = 28_000 # meters
skylines = r'E136_S8_17_skylines'
arcpy.ddd.Skyline(in_observer_point_features=road_3Dpoints,
                  out_feature_class=skylines,
                  in_surface=output_raster,
                  azimuth_increment_value_or_field=AZIMUTH_RESOLUTION_DEGREES,
                  max_horizon_radius=MAX_RADIUS,
                  use_curvature='CURVATURE',
                  use_refraction='REFRACTION')

obs_points = []
with arcpy.da.SearchCursor(road_3Dpoints, 'SHAPE@') as cursor:
    for point in cursor:
        obs_points.append(point[0])
        
print(obs_points)
print([f.name for f in arcpy.ListFields(skylines)])

with arcpy.da.SearchCursor(skylines, ['SHAPE@', 'SHAPE@LENGTH']) as cursor:
    idx = 0
    faulty_skyline_graphs = 0
    for skyline in cursor:
        print(idx, '\n')
        if skyline[1] == 0: # Check if it has valid length, some points dont work with skyline tool...?
            faulty_skyline_graphs += 1
            idx += 1
            continue
        #outTable = arcpy.CreateUniqueName(f'Graph_Test/angles_table{idx}.dbf')
        #print('Length:', skyline[1])
        arcpy.ddd.SkylineGraph(obs_points[idx],
                           skyline[0],
                           base_visibility_angle=0,
                           additional_fields    =False,
                           out_angles_table     =f'{DATA_DUMP_LOCATION}skylines/angles_table_{idx}.csv',
                           out_image_file       =f'{DATA_DUMP_LOCATION}skylines_images/graph_Test_{idx}.png'
                              )
        idx += 1
    print('Number of faulty skyline graphs:, ', faulty_skyline_graphs)


6801_50m_33.dem;6802_50m_33.dem;6900_50m_33.dem;6901_50m_33.dem;6901_50m_33.dem


[<PointGeometry object at 0x1d8948b7950[0x1d881a84f30]>, <PointGeometry object at 0x21960c1f7d0[0x1d881a85bf0]>, <PointGeometry object at 0x21960cda990[0x1d881a84d80]>, <PointGeometry object at 0x219489f1390[0x1d881a84d20]>, <PointGeometry object at 0x219489f2cd0[0x1d881a84c00]>, <PointGeometry object at 0x219489f1c50[0x1d881a85080]>, <PointGeometry object at 0x219489f1f90[0x1d881a85020]>, <PointGeometry object at 0x219489f0910[0x1d881a84f60]>, <PointGeometry object at 0x219489f3f10[0x1d881a844b0]>, <PointGeometry object at 0x219489f0a50[0x1d881a84510]>, <PointGeometry object at 0x219489f0e10[0x1d881a84240]>, <PointGeometry object at 0x21948881150[0x1d881a851a0]>, <PointGeometry object at 0x219489f0990[0x1d881a845a0]>, <PointGeometry object at 0x2194899fd10[0x1d881a843f0]>, <PointGeometry object at 0x1d88a59e990[0x1d881a84e10]>, <PointGeometry object at 0x2194899e510[0x1d881a84ba0]>, <PointGeometry object at 0x2194899e2d0[0x1d8807037e0]>, <PointGeometry object at 0x2194899f010[0x1d8807

['OID', 'Shape', 'OBSV_PT_ID', 'ORIGFTR_ID', 'Shape_Length']
0 

1 

2 



ExecuteError: ERROR 001005: The output already exists.
Failed to execute (SkylineGraph).


In [31]:
import csv
import os
import arcpy

newFile = r"C:/Users/sinhav/Documents/Kode/GDOPER_sindre/test_data/observation_points.csv"
PROJECT_LOCATION   = r'C:/Users/sinhav/Documents/ArcGIS/Projects/GDOP_preprocessing_v1/'
PROJECT_DATABASE_LOCATION = f'{PROJECT_LOCATION}GDOP_preprocessing_v1.gdb'


#arcpy.TableToTable_conversion(newFile, PROJECT_DATABASE_LOCATION, 'obs_points_GDOP.dbf')
cur = arcpy.UpdateCursor(myFeatureClass)

# Loop through the rows in the attribute table
for row in cur:
    # The variable sqMiles will get the value from the column
    # named 'Area_Miles'
    sqMiles = row.getValue('Area_Miles')
    # Calculate how many acres
    acres = (sqMiles * 640)
    # Assign the acres to a column named 'Area_Acres'
    row.setValue('Area_Acres', acres)
    # Apply the change
    cur.updateRow(row)

ExecuteError: ERROR 000600: Cannot delete C:/Users/sinhav/Documents/ArcGIS/Projects/GDOP_preprocessing_v1/GDOP_preprocessing_v1.gdb/obs_points_GDOP.  It is being edited by the application.
Failed to execute (TableToTable).


In [26]:
DATA_DUMP_LOCATION = r"C:\Users\sinhav\Documents\Kode\GDOPER_sindre\datadump_from_arcgis"
arcpy.conversion.ExportTable(in_table=observation_points,
                             out_table=DATA_DUMP_LOCATION+'\\'+observation_points + '.csv',
                             field_mapping = ["POINT_X", "POINT_Y"])

RuntimeError: Object: Error in executing tool

In [17]:
"""
arcpy.ddd.SkylineGraph(in_observer_point_features=road_3Dpoints,
                       in_line_features=skylines,
                       base_visibility_angle=0,
                       additional_fields='ADDITIONAL_FIELDS',
                       out_angles_table=table,
                       out_image_file=skylinegrap_image)

skylinegraphs = 'E136_S8_17_skylinegraph'
table = 'skyline_graphs'
skylinegraph_image = 
"""
"""
print([f.name for f in arcpy.ListFields(road_3Dpoints)])
obs_pts = []
with arcpy.da.SearchCursor(road_3Dpoints, 'SHAPE@') as cursor:
    for pnt in cursor:
        # pnt[0] are 3D point object in UTM33 (which is standard for the road source vegkart.no)
        obs_pts.append(pnt[0])
        print(pnt[0][0])

with arcpy.da.SearchCursor(skylines, 'SHAPE@') as cursor:
    idx = 0
    faulty_skyline_graphs = 0
    for skyline in cursor:
        print(idx, obs_pts, len(obs_pts))
        if idx != 8:
            idx += 1
            continue
        #print(row[0])
        #outTable = arcpy.CreateUniqueName(f'Graph_Test/angles_table{idx}.dbf')
        result = arcpy.ddd.SkylineGraph(
                           obs_pts[idx],
                           skyline,
                           base_visibility_angle=0,
                           additional_fields=False,
                           out_angles_table=f'C:/Users/sinhav/Documents/ArcGIS/Projects/GDOP_preprocessing_v1/Graph_Test/Graph_Test/skyline_angles_{idx}.dbf',
                           out_image_file=  f'C:/Users/sinhav/Documents/ArcGIS/Projects/GDOP_preprocessing_v1/Graph_Test/Graph_Test/skyline_angles_{idx}.png')
        # 0 degree is north, and 90 degree is east
        print(idx, type(result.getOutput(2)), result.getOutput(2), '|', obs_pts[idx][0])
        if result.getOutput(2) == '-nan(ind)':
            faulty_skyline_graphs += 1
        idx += 1
    print('Number of faulty skyline graphs:, ', faulty_skyline_graphs)
"""

"\nprint([f.name for f in arcpy.ListFields(road_3Dpoints)])\nobs_pts = []\nwith arcpy.da.SearchCursor(road_3Dpoints, 'SHAPE@') as cursor:\n    for pnt in cursor:\n        # pnt[0] are 3D point object in UTM33 (which is standard for the road source vegkart.no)\n        obs_pts.append(pnt[0])\n        print(pnt[0][0])\n\nwith arcpy.da.SearchCursor(skylines, 'SHAPE@') as cursor:\n    idx = 0\n    faulty_skyline_graphs = 0\n    for skyline in cursor:\n        print(idx, obs_pts, len(obs_pts))\n        if idx != 8:\n            idx += 1\n            continue\n        #print(row[0])\n        #outTable = arcpy.CreateUniqueName(f'Graph_Test/angles_table{idx}.dbf')\n        result = arcpy.ddd.SkylineGraph(\n                           obs_pts[idx],\n                           skyline,\n                           base_visibility_angle=0,\n                           additional_fields=False,\n                           out_angles_table=f'C:/Users/sinhav/Documents/ArcGIS/Projects/GDOP_preprocessing_

In [35]:
"""
import arcpy
arcpy.env.workspace = r"C:\Users\sinhav\Documents\ArcGIS\Projects\Skyline_funker\Skyline_funker.gdb"

def print_feature_classes_and_lengths():
    # Print alle objekter med tilhorende lengde
    fc_list = arcpy.ListFeatureClasses()
    print(fc_list)
    for fc in fc_list:
        count = arcpy.GetCount_management(fc)
        print(fc, count)
    
def create_skyline_Graph_from_points():
    return


fc = 'database.gdb/feature'
columns = ['column1', 'column2']

results = []

with arcpy.da.SearchCursor(fc, columns) as cursor:
    for row in cursor:
        results.append([row[0], row[1]])

in_observer_point_features = r'c6901_50m_33_InterpolateShap_Dissolve'
in_line_features           = r'c6901_50m_33_Interpo_Skyline'
print([f.name for f in arcpy.ListFields(in_observer_point_features)])
points = []
with arcpy.da.SearchCursor(in_observer_point_features, 'SHAPE@') as cursor:
    for row in cursor:
        # row[0] are 3D point object in UTM33 (which is standard for the road source vegkart.no)
        points.append(row[0])
        
    #print(points)

with arcpy.da.SearchCursor(in_line_features, 'SHAPE@') as cursor:
    idx = 0
    faulty_skyline_graphs = 0
    for row in cursor:
        if idx > 80: continue
        #print(row[0])
        #outTable = arcpy.CreateUniqueName(f'Graph_Test/angles_table{idx}.dbf')
        outTable = f'Graph_Test/angles_table{idx}.dbf'
        result = arcpy.ddd.SkylineGraph(points[idx],
                           row,
                           base_visibility_angle=0,
                           additional_fields=False,
                           out_angles_table=outTable,
                           out_image_file=f'Graph_Test/Graph_Test_{idx}.png')
        # 0 degree is north, and 90 degree is east
        print(idx, type(result.getOutput(2)), result.getOutput(2), '|', points[idx][0])
        if result.getOutput(2) == '-nan(ind)':
            faulty_skyline_graphs += 1
        idx += 1
    print('Number of faulty skyline graphs:, ', faulty_skyline_graphs)


arcpy.ddd.SkylineGraph(in_observer_point_features,
                           in_line_features,
                           {base_visibility_angle},
                           {additional_fields},
                           {out_angles_table},
                           {out_graph},
                           {out_image_file})
                           
points = []
with arcpy.da.SearchCursor(in_observer_point_features, 'SHAPE@') as cursor:
    for row in cursor:
        points.append(row[0][0])
    print(points)


# Lag ppt presentasjon
# Noter future work på presenetasjon

"""

RuntimeError: cannot open 'database.gdb/feature'

In [ ]:
"""
WKID_XY_WGS_1984 = 4326
WKID_Z_WGS_1984 = 115700
out_coordinate_system = arcpy.SpatialReference(WKID_XY_WGS_1984, WKID_Z_WGS_1984)
"""

In [44]:
# Coordinate system error. kl 00.08, 26.09.2024.
import arcpy

PROJECT_LOCATION = r'C:/Users/sinhav/Documents/ArcGIS/Projects/GDOP_preprocessing_v1/'
PROJECT_DATABASE_LOCATION = f'{PROJECT_LOCATION}GDOP_preprocessing_v1.gdb'

arcpy.env.workspace = PROJECT_DATABASE_LOCATION
# The .tif ouput_raster got different coordinat system somehow,
# and therefore skyline tool not working,
# and this code is not used.
WKID_XY_WGS_1984 = 32633 #"WGS 1984 UTM Zone 33N"# 4326
#WKID_Z_WGS_1984 = 115700
out_coordinate_system = arcpy.SpatialReference(WKID_XY_WGS_1984)#, WKID_Z_WGS_1984)
#arcpy.env.outputCoordinateSystem = out_coordinate_system

road_shp = 'E136_S8_17'
road_dis = 'E136_S8_17_dis'

# Merge all roadobjects into one
arcpy.management.Dissolve(road_shp, road_dis)

DEM = ['6801_50m_33.dem', '6802_50m_33.dem',
       '6900_50m_33.dem', '6901_50m_33.dem',
       '6901_50m_33.dem']

input_rasters = ''.join([x+";" for x in DEM])[:-1]
output_raster = r'dem50.tif'
print(input_rasters)

#arcpy.management.MosaicToNewRaster
#arcpy.management.MosaicToNewRaster
PROJECT_DATABASE_LOCATION = r'C:\Users\sinhav\Documents\ArcGIS\Projects\GDOP_preprocessing_v1'
arcpy.MosaicToNewRaster_management(input_rasters=input_rasters,
                                   output_location=PROJECT_DATABASE_LOCATION,
                                   raster_dataset_name_with_extension=output_raster,
                                   pixel_type="16_BIT_SIGNED",
                                   number_of_bands="1")


road_2Dpoints = 'E136_S8_17_pts_2D'
SPACING = 5_000 # meters

# Create spaced points along roadcurve
arcpy.management.GeneratePointsAlongLines(Input_Features=road_dis,
                                          Output_Feature_Class=road_2Dpoints,
                                          Point_Placement='DISTANCE',
                                          Distance=f'{SPACING} meters')

road_3Dpoints = r'E136_S8_17_pts_3D'
arcpy.ddd.InterpolateShape(in_surface=output_raster,
                           in_feature_class=road_2Dpoints,
                           out_feature_class=road_3Dpoints)

#observation_points = r'E136_S8_17_pts_3D_WGS84_3D'
#arcpy.Project_management(road_3Dpoints, observation_points, out_coordinate_system)

#arcpy.AddXY_management(observation_points)

#out_coordinate_system2 = arcpy.SpatialReference(item=output_raster) #4326)#, WKID_Z_WGS_1984)
#arcpy.Project_management(road_3Dpoints, observation_points, out_coordinate_system2)


MAX_RADIUS = 28_000 # meters
skylines = r'E136_S8_17_skylines'
arcpy.ddd.Skyline(in_observer_point_features=road_3Dpoints,
                  out_feature_class=skylines,
                  in_surface=output_raster,
                  max_horizon_radius=MAX_RADIUS,
                  use_curvature='CURVATURE',
                  use_refraction='REFRACTION')

WKID_XY_WGS_1984 = 4326
WKID_Z_WGS_1984 = 115700
out_coordinate_system = arcpy.SpatialReference(WKID_XY_WGS_1984, WKID_Z_WGS_1984)
observation_points = r'E136_S8_17_pts_3D_obs'
arcpy.Project_management(road_3Dpoints, observation_points, out_coordinate_system)
arcpy.AddXY_management(observation_points)

print(road_3Dpoints)

obs_points = []
with arcpy.da.SearchCursor(road_3Dpoints, 'SHAPE@') as cursor:
    for point in cursor:
        # row[0] are 3D point object in UTM33 (which is standard for the road source vegkart.no)
        obs_points.append(point[0])
        
print(obs_points)
print([f.name for f in arcpy.ListFields(skylines)])

with arcpy.da.SearchCursor(skylines, ['SHAPE@', 'SHAPE@LENGTH']) as cursor:
    idx = 0
    faulty_skyline_graphs = 0
    for skyline in cursor:
        if skyline[1] == 0: # Check if it has valid length
            faulty_skyline_graphs += 1
        #print(skyline[0])
        #outTable = arcpy.CreateUniqueName(f'Graph_Test/angles_table{idx}.dbf')
        print('Length:', skyline[1])
        arcpy.ddd.SkylineGraph(obs_points[idx],
                           skyline[0],
                           base_visibility_angle=0,
                           additional_fields    =False,
                           out_angles_table     =f'{PROJECT_LOCATION}Graph_Test/angles_table{idx}.csv',
                           out_image_file       =f'{PROJECT_LOCATION}Graph_Test/graph_Test_{idx}.png')
        # 0 degree is north, and 90 degree is east
        #print(idx, type(result.getOutput(2)), result.getOutput(2), '|', obs_points[idx][0])
        idx += 1
    print('Number of faulty skyline graphs:, ', faulty_skyline_graphs)


6801_50m_33.dem;6802_50m_33.dem;6900_50m_33.dem;6901_50m_33.dem;6901_50m_33.dem
E136_S8_17_pts_3D
[<PointGeometry object at 0x28449d1c450[0x2844845a220]>, <PointGeometry object at 0x28449d1c550[0x2844845b090]>, <PointGeometry object at 0x28449d1e910[0x2844845b1b0]>, <PointGeometry object at 0x28449d1dc90[0x2844845a490]>, <PointGeometry object at 0x28449d1f750[0x2844845b930]>, <PointGeometry object at 0x28449d57890[0x2844845bf30]>, <PointGeometry object at 0x2844882f1d0[0x2844845af40]>, <PointGeometry object at 0x28449d1d110[0x2844845bcc0]>, <PointGeometry object at 0x28449d1f9d0[0x284484597a0]>, <PointGeometry object at 0x28449d1cb90[0x2844845a7f0]>, <PointGeometry object at 0x28449d1c0d0[0x2844845a580]>, <PointGeometry object at 0x28449d1f950[0x2844845a760]>, <PointGeometry object at 0x28449d1c890[0x28448459bf0]>, <PointGeometry object at 0x28449d1d650[0x2844845aeb0]>, <PointGeometry object at 0x28449d1ef10[0x2844845b390]>, <PointGeometry object at 0x28449d1cc10[0x2844845b870]>, <Poin

ExecuteError: ERROR 001005: The output already exists.
Failed to execute (SkylineGraph).


In [ ]:
""" Worked kl 20.40, date:26.09.2024.

import arcpy

PROJECT_LOCATION = r'C:/Users/sinhav/Documents/ArcGIS/Projects/GDOP_preprocessing_v1/'
PROJECT_DATABASE_LOCATION = f'{PROJECT_LOCATION}GDOP_preprocessing_v1.gdb'

arcpy.env.workspace = PROJECT_DATABASE_LOCATION

road_shp = 'E136_S8_17'
road_dis = 'E136_S8_17_dis'

# Merge all roadobjects into one
arcpy.management.Dissolve(road_shp, road_dis)

DEM = ['6801_50m_33.dem', '6802_50m_33.dem',
       '6900_50m_33.dem', '6901_50m_33.dem',
       '6901_50m_33.dem']

input_rasters = ''.join([x+";" for x in DEM])[:-1]
output_raster = r'dem50.tif'
print(input_rasters)

#arcpy.management.MosaicToNewRaster
#arcpy.management.MosaicToNewRaster
PROJECT_DATABASE_LOCATION = r'C:\Users\sinhav\Documents\ArcGIS\Projects\GDOP_preprocessing_v1'
arcpy.MosaicToNewRaster_management(input_rasters=input_rasters,
                                   output_location=PROJECT_DATABASE_LOCATION,
                                   raster_dataset_name_with_extension=output_raster,
                                   pixel_type="16_BIT_SIGNED",
                                   number_of_bands="1")


road_2Dpoints = 'E136_S8_17_pts_2D'
SPACING = 5_000 # meters

# Create spaced points along roadcurve
arcpy.management.GeneratePointsAlongLines(Input_Features=road_dis,
                                          Output_Feature_Class=road_2Dpoints,
                                          Point_Placement='DISTANCE',
                                          Distance=f'{SPACING} meters')

road_3Dpoints = r'E136_S8_17_pts_3D'
arcpy.ddd.InterpolateShape(in_surface=output_raster,
                           in_feature_class=road_2Dpoints,
                           out_feature_class=road_3Dpoints)

observation_points = r'new_E136_S8_17_pts_3D'
out_coordinate_system2 = arcpy.SpatialReference(4326)#, WKID_Z_WGS_1984)
arcpy.Project_management(road_3Dpoints, observation_points, out_coordinate_system2)
arcpy.AddXY_management(observation_points)


MAX_RADIUS = 28_000 # meters
skylines = r'E136_S8_17_skylines'
arcpy.ddd.Skyline(in_observer_point_features=road_3Dpoints,
                  out_feature_class=skylines,
                  in_surface=output_raster,
                  max_horizon_radius=MAX_RADIUS,
                  use_curvature='CURVATURE',
                  use_refraction='REFRACTION')



print(road_3Dpoints)

obs_points = []
with arcpy.da.SearchCursor(road_3Dpoints, 'SHAPE@') as cursor:
    for point in cursor:
        # row[0] are 3D point object in UTM33 (which is standard for the road source vegkart.no)
        obs_points.append(point[0])
        
print(obs_points)
print([f.name for f in arcpy.ListFields(skylines)])

with arcpy.da.SearchCursor(skylines, ['SHAPE@', 'SHAPE@LENGTH']) as cursor:
    idx = 0
    faulty_skyline_graphs = 0
    for skyline in cursor:
        if skyline[1] == 0: # Check if it has valid length
            faulty_skyline_graphs += 1
        #print(skyline[0])
        #outTable = arcpy.CreateUniqueName(f'Graph_Test/angles_table{idx}.dbf')
        print('Length:', skyline[1])
        arcpy.ddd.SkylineGraph(obs_points[idx],
                           skyline[0],
                           base_visibility_angle=0,
                           additional_fields    =False,
                           out_angles_table     =f'{PROJECT_LOCATION}Graph_Test/angles_table{idx}.csv',
                           out_image_file       =f'{PROJECT_LOCATION}Graph_Test/graph_Test_{idx}.png')
        # 0 degree is north, and 90 degree is east
        #print(idx, type(result.getOutput(2)), result.getOutput(2), '|', obs_points[idx][0])
        idx += 1
    print('Number of faulty skyline graphs:, ', faulty_skyline_graphs)
    

 
"""

In [19]:
DATA_DUMP_LOCATION = r"C:\Users\sinhav\Documents\Kode\GDOPER_sindre\datadump_from_arcgis"
arcpy.conversion.ExportTable(in_table=observation_points,
                             out_table=DATA_DUMP_LOCATION+'\\'+observation_points + '.csv'
                            )

<Result 'C:\\Users\\sinhav\\Documents\\Kode\\GDOPER_sindre\\datadump_from_arcgis\\new_E136_S8_17_pts_3D.csv'>

In [18]:
print(5)

5
